In [18]:
# Download CBDB sqlite
!git clone https://github.com/cbdb-project/cbdb_sqlite.git

fatal: destination path 'cbdb_sqlite' already exists and is not an empty directory.


In [19]:
import glob
import os
list_of_files = glob.glob('./cbdb_sqlite/*.7z') 
latest_file = max(list_of_files, key=lambda x : os.path.basename(x).split('_')[1])
print(latest_file)

./cbdb_sqlite/CBDB_20210525.7z


In [20]:
# Explode latest database file
!pip install pyunpack
!pip install patool
from pyunpack import Archive
Archive(latest_file).extractall(".")

In [21]:
# Create connection to database
import sqlite3
import pandas as pd
global CONN
database_file = os.path.basename(latest_file).split(".")[0]+".db"
CONN = sqlite3.connect(database_file)

In [22]:
fullenames_df = pd.read_sql_query("SELECT c_name_chn FROM BIOG_MAIN", CONN).drop_duplicates().dropna()
# exclude c_alt_name_type_code = 7 (birth-order name / 行第, e.g. 第二十三)
givennames_df = pd.read_sql_query("SELECT c_alt_name_chn FROM ALTNAME_DATA where c_alt_name_type_code <> 7", CONN).drop_duplicates().dropna()

In [23]:
def names_massage(df, column):
  # remove content in brackets
  df[column] = df[column].str.replace("\(.*?\)", "", regex=True)
  df[column] = df[column].str.replace("\（.*?\）", "", regex=True)
  # remove single-character name
  df = df[df[column].apply(lambda x: len(x)>1)]
  # filter out weird characters
  df = df[df[column].str.contains('\?|？|#|!|！|【|。|、.|,|，|\*|、|\[|□|�|{|「')==False]
  df = df[df[column].str.contains('.*[a-zA-Z]+.*')==False]
  df = df[df[column].str.contains('\d')==False]
  df.drop_duplicates().dropna()
  return df

In [24]:
filtered_fullnames = names_massage(fullenames_df, 'c_name_chn')
filtered_givennames = names_massage(givennames_df, 'c_alt_name_chn')

In [35]:
filtered_fullnames

,c_name_chn
0,富興
1,富陞額
2,嵩山
3,得祿
4,恆善
...,...
491162,龔龍見
491164,郎侃
491165,郎崇賢
491166,郎汝璠


In [42]:
filtered_givennames

,c_alt_name_chn
0,處厚
1,厚卿
2,湛然
3,元章
4,寳臣
...,...
139602,南阿山人
139606,明徵
139610,爾思
139611,飛虎將軍


In [55]:
# combine df
final_df = pd.concat([filtered_fullnames, filtered_givennames], axis=1).stack().reset_index(drop=True).drop_duplicates().dropna().to_frame(name='name')

In [56]:
final_df

,name
0,富興
1,處厚
2,富陞額
3,厚卿
4,嵩山
...,...
424953,龔龍見
424954,郎侃
424955,郎崇賢
424956,郎汝璠


In [57]:
final_df.to_csv('cbdb_entity_names.csv', index=False)